In [1]:
import csv 
import numpy as np
import matplotlib.pyplot as plt
import math
import ctypes as ct
import scipy.interpolate
from aerofiles.igc import Reader

from scipy.optimize import minimize
from IPython.display import display, clear_output

import plotly.graph_objects as go

import pandas as pd

import concurrent.futures
import multiprocessing
import os



In [2]:
# Function to read airfoil data at each station
 
def GetAirfoilData(Filename):
    with open(Filename, 'r') as infile:
        AirfoilData = np.loadtxt(infile, dtype=str, skiprows=0, unpack=True)
    
    return AirfoilData

In [3]:
# Function to read wing geometry

def GetWingData(Filename):
    with open(Filename, 'r') as infile:
        WingData = np.loadtxt(infile, dtype=float, skiprows=0, unpack=True)
    area = 0
    span = 0
    
    for i in range(0,len(WingData[0]),1):
        span = span + WingData[2][i]
        area = area + ((WingData[0][i] + WingData[1][i])/2)*WingData[2][i]
    
    AR = 2*(span**2)/area
    
    return area, span,AR,WingData
    



# CALL XFOIL

In [9]:
def CallXfoilcl(Cl, Re, AirfoilName, Flap,xpos,ypos):

    
    mydll = ct.cdll.LoadLibrary("../XFOIL/XFOIL.dll")

    if os.path.exists(AirfoilName) == True:
        ISCONV = ct.c_bool(False)
        RE_IN = ct.c_double(Re)
        CL_IN = ct.c_double(Cl)
    
        array = (ct.c_double * 11) (0,0,0,0,0,0,0,Flap,xpos,ypos,0)
        
        l = ct.c_int(0)
        my_string = AirfoilName.encode() # Convert string to bytes
        l = len(my_string)
        _= mydll.xfoil_cl(ct.byref(array), ct.byref(CL_IN),ct.byref(RE_IN),ct.byref(ISCONV),ct.c_char_p(my_string),ct.c_int(l))

                # Array1(1) = ADEG
                # Array1(2) = CD
                # Array1(3) = CDF
                # Array1(4) = CM
                # Array1(5) = HMOM
                # Array1(6) = HFX
                # Array1(7) = HFY
        if ISCONV.value == False:
            print(AirfoilName,Re,Cl,array[1])         #[uncomment if you want to see whether iteration for station has converged]
        return array[0],array[1],array[3],ISCONV.value
    else:
        print("Airfoil Filename does not exist->" + str(AirfoilName))
        return(1,1,1,False)


In [ ]:
AOA, Cd,Cm,ISCONV = CallXfoilcl(0.5,1000000,'./AIRFOILS/T12',0,0.86,0)
print(AOA)

In [5]:
def Get_Chord(WingData,y_pos):
    chord = 0
    number_of_panels = WingData[0].size

    #determine wing span from wing file
    span = 0
    if number_of_panels > 1:
        for panel_length in WingData[2]:
            span = span + panel_length 
    else:
        span = WingData[2]
        
    #print("Wing half span => " + str(span))
    
    
   
   
    current_panel = 0
    current_span_pos = WingData[2][0]   #outer section of current panel
     
    while current_panel <= number_of_panels+1:
        if y_pos < current_span_pos:
                chord = ((WingData[0][current_panel] - WingData[1][current_panel]) / WingData[2][current_panel]) * (current_span_pos - y_pos) + WingData[1][current_panel]
                break
        
        current_panel += 1 
        current_span_pos = current_span_pos + WingData[2][current_panel]
               
        #print(current_panel,current_span_pos)
        
    
    
    #print(np.stack((np.linspace(0,span,n),chords),axis=1))    
    return(chord)

In [6]:
def Solve_LL(wingdata,span,AR,alpha,a,n):
    
    #a is the lift slope cl/radians  -> 2*PI
    
    theta_range = np.linspace(90,0.01,n)
    AMatrix = np.ones(shape=(n,n))
    X = np.ones(n)
    delta = 0
    Cl = np.zeros(n)
    y_pos = np.zeros(n)
    
    
    i = 0
    for theta in theta_range:
        unit_y = -math.cos(np.deg2rad(theta)) 
        c = Get_Chord(wingdata,-unit_y*span)
        mu = c*a/(8*span)
        
        X[i] = mu*(np.deg2rad(alpha))*(math.sin(np.deg2rad(theta)))
        for j in range(0,n,1):
            AMatrix[i][j] = (math.sin(((j+1)*2 -1)*np.deg2rad(theta)))*(((j+1)*2-1)*mu + (math.sin(np.deg2rad(theta))))
        
        y_pos[i] = -unit_y*span
        i+=1
    
    #print(A)
    #print(X)      
    #A = np.matmul(np.linalg.inv(AMatrix),X)
    A = np.linalg.solve(AMatrix, X)
    
    
    for i in range(1,n,1):
        delta = delta +((i+1)*2-1)*(A[i]**2)/(A[0]**2)

    #e =  (1/(1+delta))     

    k = 0
    for theta in theta_range:  
        for j in range(0,n,1):
            unit_y = -math.cos(np.deg2rad(theta))
            c = Get_Chord(wingdata,-unit_y*span)
            Cl[k] = Cl[k]+ (8*span)*(A[j]*(math.sin(((j+1)*2-1)*np.deg2rad(theta)))) / c
            
        k+=1
           
    CL = A[0]*(np.pi)*AR
    Cl_unit = Cl/CL
    y_pos[0] = 0.0
    return(Cl_unit,y_pos,CL,delta) 
          
    

# AIRFOIL ANALYSIS


In [10]:
# AIRFOIL ANALYSIS II PARALLEL PROCESSING   (FLAP AUTO RANGES - BEST FLAP)

n = 60
rho = 1.225
mu = 1.789e-5
#V_inf = 40         #[m/s]
alpha = 5          # angle of attack (only for Lifting line solver) [deg]
a = 2*np.pi        # lift curve slope
Weight = 550*9.81  #[N]
Bank_Angle = 0     #[deg]
Flap = 0           #[0 degrees is neutral or no flap]
Pressure_altitude = 0  #[m]
OAT = 15  #[Deg]



Area,Span,AR, WingData = GetWingData("JS4_wing_15m.dat")
ChordData  = GetAirfoilData("JS4_Airfoil_Data.dat")  


Airfoil_foldername = "./AIRFOILS/SC/"
with open("AIRFOIL_ANALYSIS_SC.dat", 'r') as infile:
     Airfoils = np.loadtxt(infile, dtype=str, skiprows=0, unpack=True)

        



with open("SPEED_SETTINGS_SC.dat", 'r') as infile:
     SPEEDFLAP = np.loadtxt(infile, dtype=str, skiprows=0, unpack=True)


cl,y_pos,CL_theory,delta = Solve_LL(WingData,Span,AR,alpha,a,n)   
#set cl interpolation function 
cl_interp = scipy.interpolate.interp1d(y_pos,cl)


Station = 4                                             #  STATION

span = 0
for i in range(0,Station-1,1):
    span = span + 0.995*WingData[2][i]
print(span)



filename1 = "Airfoil_analysis.csv"
filename2 = "Airfoil_flap_vs_drag.csv"
csvfile1 = open(filename1, 'w')
csvfile2 = open(filename2, 'w')
csvwriter1 = csv.writer(csvfile1)
csvwriter2 = csv.writer(csvfile2)

csvheader1 = Airfoils[0]
csvheader1 = np.insert(csvheader1, 0,"CL")

csvheader2 = []

#print(csvheader)
csvwriter1.writerow(csvheader1)   



num_processes = multiprocessing.cpu_count()
print(num_processes)

step = 1

for k in range(0,len(SPEEDFLAP[0]),1):
    
    i = Station-1
    Airfoil_min = "none"
    flapsetting = "99"
    V_inf = float(SPEEDFLAP[0][k])/3.6
    Re = rho*V_inf*float(ChordData[1][i])/mu
    #print(V_inf,Re)
    CL = (2*Weight) /  (rho*(V_inf)**2  *2*Area*np.cos(np.deg2rad(Bank_Angle)))    

    print("Station->"+str(i+1) + "  " + str(SPEEDFLAP[0][k])+"km/h  " + "CL = "+str(cl_interp(span)*CL) + " RE = " + str(Re))
 
    ISCONV = True
    cd_collector = []
    rowlist = []
    flaplist = []
    m = np.arange(float(SPEEDFLAP[1][k]),float(SPEEDFLAP[2][k])+step,step)
    
    csvheader2 = []
    
    for v in range(0,len(m),1):
        csvheader2.append(str(m[v]))
    
    csvheader2.insert(0,"FLAP")
    csvheader2.append('')
    csvheader2.append('')
    csvheader2.append('')
    csvwriter2.writerow(csvheader2) 
    csvheader2.clear()
    
    
    rowlist.append(float(cl_interp(span)*CL))
    
    for j in range(0,len(Airfoils[0]),1):
        
        Cd_min_airfoil = 0.1
        Flap_min = "-99"
     
        cdlist = [1]*len(m)
        
        
        def min_drag(fl):
            xpos = float(Airfoils[1][j])
            ypos = float(Airfoils[2][j])
            AOA, Cd,Cm,ISCONV = CallXfoilcl(cl_interp(span)*CL,Re,Airfoil_foldername + Airfoils[0][j],fl,xpos,ypos)
            #Cd = 0.005
            if ISCONV == True:
                return Airfoils[0][j],'{:.5f}'.format(Cd), fl
            else:
                return Airfoils[0][j],1,1 
        
        #cdlist = np.insert(cdlist, 0,Airfoils[0][j])
         
        
        with multiprocessing.Pool(processes=num_processes) as pool:
            cd_collector = pool.map(min_drag,m)        
          
        
        a = []
        
        for l in range(0,len(m),1):
            a.append(str(cd_collector[l][1]))
            if float(cd_collector[l][1]) < Cd_min_airfoil:
                Cd_min_airfoil = float(cd_collector[l][1])
                Flap_min = cd_collector[l][2]
        
        
        a.insert(0,Airfoils[0][j])
        csvwriter2.writerow(a) 
        print(Airfoils[0][j],Cd_min_airfoil,Flap_min)    #uncomment to print details
        
        #print(cd_collector)
        #print(Airfoils[0][j],min(cd_collector)) 
        #print(flaplist)
            
        
        rowlist.append(Cd_min_airfoil)  
        flaplist.append(Flap_min)    #not written to csv file currently
  
    # print(rowlist)
    # print(flaplist)
    csvwriter1.writerow(rowlist) 
    
csvfile1.close()
csvfile2.close()
print("Done")       
    

5.950100000000001
128
Station->4  100km/h  CL = 1.1359962049842331 RE = 1017599.8385193468
STmod11 0.00849 0.0
OPT110 0.00793 0.0
ST1 0.0082 0.0
BS1 0.00903 0.0
Airfoil_Station_1_smooth.dat 0.00774 0.0
Airfoil_Station_2_smooth.dat 0.00993 0.0
Airfoil_Station_3_smooth.dat 0.01021 0.0
Airfoil_Station_4_smooth.dat 0.00995 0.0
Airfoil_Station_5_smooth.dat 0.01167 0.0
Airfoil_Station_6_smooth.dat 0.01147 0.0
Station->4  130km/h  CL = 0.6721871035409666 RE = 1322879.7900751506
STmod11 0.00578 0.0
OPT110 0.00562 0.0
ST1 0.00604 0.0
BS1 0.0061 0.0
Airfoil_Station_1_smooth.dat 0.00584 0.0
Airfoil_Station_2_smooth.dat 0.00559 0.0
Airfoil_Station_3_smooth.dat 0.00551 0.0
Airfoil_Station_4_smooth.dat 0.00557 0.0
Airfoil_Station_5_smooth.dat 0.00542 0.0
Airfoil_Station_6_smooth.dat 0.00611 0.0
Station->4  160km/h  CL = 0.4437485175719661 RE = 1628159.741630955
STmod11 0.00497 0.0
OPT110 0.00503 0.0
ST1 0.00501 0.0
BS1 0.00493 0.0
Airfoil_Station_1_smooth.dat 0.00562 0.0
Airfoil_Station_2_smooth.dat

In [14]:
# AIRFOIL ANALYSIS        (FIXED FLAP from file)


with open("SPEED_SETTINGS_SC.dat", 'r') as infile:
     SPEEDFLAP = np.loadtxt(infile, dtype=str, skiprows=0, unpack=True)

    
n = 60 
rho = 1.225
mu = 1.789e-5
#V_inf = 40         #[m/s]
alpha = 5          # angle of attack (only for Lifting line solver) [deg]
a = 2*np.pi        # lift curve slope
Weight = 500*9.81  #[N]
Bank_Angle = 0     #[deg]
Flap = 0           #[0 degrees is neutral or no flap]
Pressure_altitude = 0  #[m]
OAT = 15  #[Deg]


Area,Span,AR, WingData = GetWingData("JS4_wing_15m.dat")
ChordData  = GetAirfoilData("JS4_Airfoil_Data.dat")



cl,y_pos,CL_theory,delta = Solve_LL(WingData,Span,AR,alpha,a,n)   
#set cl interpolation function 
cl_interp = scipy.interpolate.interp1d(y_pos,cl)
    
    
Airfoil_foldername = "./AIRFOILS/SC/"
with open("AIRFOIL_ANALYSIS_SC.dat", 'r') as infile:
     Airfoils = np.loadtxt(infile, dtype=str, skiprows=0, unpack=True)

        

filename = "Airfoil_analysis.csv"


Station = 1                                              #  STATION

span = 0
for i in range(0,Station-1,1):
    span = span + 0.995*WingData[2][i]
print("Span pos->"+str(span))


csvfile = open(filename, 'w')
csvwriter = csv.writer(csvfile) 



csvheader = Airfoils[0]
csvheader = np.insert(csvheader, 0,"CL")

print(csvheader)
csvwriter.writerow(csvheader)   

num_processes = multiprocessing.cpu_count()
print(num_processes)





for k in range(0,len(SPEEDFLAP[0]),1):
    
    i = Station-1
    Airfoil_min = "none"

    V_inf = float(SPEEDFLAP[0][k])/3.6
    Re = rho*V_inf*float(ChordData[1][i])/mu
    #print(V_inf,Re)
    CL = (2*Weight) /  (rho*(V_inf)**2  *2*Area*np.cos(np.deg2rad(Bank_Angle)))    
    
    m = float(SPEEDFLAP[3][k])    #fixed flap setting from file 
    
    print("Station->"+str(i+1) + "  " + str(SPEEDFLAP[0][k])+"km/h  " + "Cl = "+str(cl_interp(span)*CL) + " RE = " + str(Re) + " FLAP =" + str(m))
 
    ISCONV = True
    cd_collector = []
    rowlist = []
    flaplist = []
    
    rowlist.append(float(cl_interp(span)*CL))
    for j in range(0,len(Airfoils[0]),1):
        step = 1
        Cd_min_airfoil = 0.1
        Flap_min = "-99"
        
       
        xpos = float(Airfoils[1][j])
        ypos = float(Airfoils[2][j])
        AOA, Cd,Cm,ISCONV = CallXfoilcl(cl_interp(span)*CL,Re,Airfoil_foldername + Airfoils[0][j],m,xpos,ypos)
#         if ISCONV == True:
#             return Airfoils[0][j],'{:.5f}'.format(Cd), m
#         else:
#             print("not converged")
#             return Airfoils[0][j],1,1 
    

        # with multiprocessing.Pool(processes=num_processes) as pool:
        #     cd_collector = pool.map(min_drag,m)        
        
        
        # for l in range(0,len(m),1):
        #     if float(cd_collector[l][1]) < Cd_min_airfoil:
        #         Cd_min_airfoil = float(cd_collector[l][1])
        #         Flap_min = cd_collector[l][2]
                
        
        #print(Airfoils[0][j],Cd_min_airfoil,Flap_min)    #uncomment to print details
        
        if ISCONV == True:
            print(Airfoils[0][j],'{:.5f}'.format(Cd))
            rowlist.append(Cd) 
        else:
            print(Airfoils[0][j],'Not converged')
            rowlist.append(' ')
             
         
        flaplist.append(Flap_min)    #not written to csv file currently
  
    # print(rowlist)
    # print(flaplist)
    csvwriter.writerow(rowlist) 
    
csvfile.close()
print("Done")       
    

Span pos->0
['CL' 'STmod11' 'OPT110' 'ST1' 'BS1' 'Airfoil_Station_1_smooth.dat'
 'Airfoil_Station_2_smooth.dat' 'Airfoil_Station_3_smooth.dat'
 'Airfoil_Station_4_smooth.dat' 'Airfoil_Station_5_smooth.dat'
 'Airfoil_Station_6_smooth.dat']
128
Station->1  100km/h  Cl = 1.0413657514164862 RE = 1559685.7338053535 FLAP =0.0
STmod11 0.00675
OPT110 0.00633
ST1 0.00669
BS1 0.00798
Airfoil_Station_1_smooth.dat 0.00656
Airfoil_Station_2_smooth.dat 0.00781
Airfoil_Station_3_smooth.dat 0.00752
Airfoil_Station_4_smooth.dat 0.00783
Airfoil_Station_5_smooth.dat 0.00745
Airfoil_Station_6_smooth.dat 0.00934
Station->1  130km/h  Cl = 0.6161927523174476 RE = 2027591.4539469597 FLAP =0.0
STmod11 0.00482
OPT110 0.00472
ST1 0.00485
BS1 0.00525
Airfoil_Station_1_smooth.dat 0.00519
Airfoil_Station_2_smooth.dat 0.00484
Airfoil_Station_3_smooth.dat 0.00474
Airfoil_Station_4_smooth.dat 0.00482
Airfoil_Station_5_smooth.dat 0.00464
Airfoil_Station_6_smooth.dat 0.00548
Station->1  160km/h  Cl = 0.4067834966470649 

In [135]:
# SAMPLE CODE   (RUN XFOIL FOR SINGLE CASE)

CL = 0.643
Re = 1693780
Airfoil_foldername = "./AIRFOILS/"
Airfoilname = "L7"
Flap = 3
xpos = 0.85
ypos = 0.0

AOA, Cd,Cm,ISCONV = CallXfoilcl(CL,Re,Airfoil_foldername + Airfoilname,Flap,xpos,ypos)

print('{:.5f}'.format(Cd))

0.004128888249397278


In [12]:
# MAKE A CHART FROM MINIMUM AIRFOIL CD VALUES

plt.rcParams["figure.figsize"] = [9.50, 5.50]
plt.rcParams["figure.autolayout"] = True

#markers = ['o', 's', 'D', 'p', '*', 'h']
#df.plot(x='X', y='Y', marker=markers, linestyle='-', figsize=(8, 6))




df = pd.read_csv('Airfoil_analysis.csv')
df.set_index('CL').plot(marker = ".")
#plt.show()
plt.ylim(0.004, 0.012)

EmptyDataError: No columns to parse from file

In [ ]:
#SAMPLE CODE

Area,Span,AR, WingData = GetWingData("JS6_wing_18m.dat")
print(WingData)
print("Number of panels => " + str(WingData[0].size))
print("Aspect Ratio => " + str(AR))
print("Area => " + str(Area*2))
print("Span => " + str(Span*2))
#print(dir(WingData))